# Influence of temporal resolution

- /Users/hkromer/02_PhD/02_Data/01_COMSOL/01_IonOptics/02.current_chamber/02.MR/temporal_refinement/data/temporal_resolution/time_outputs/02.MR.102.temporal_refinement.save_1e8.001.csv

- file 02.MR.102.temporal_refinement.save_1e8.001.csv: 2e-8
- file 02.MR.102.temporal_refinement.save_1e8.002.csv: 2e-9
- file 02.MR.102.temporal_refinement.save_1e8.003.csv: 2e-10
- file 02.MR.102.temporal_refinement.save_1e8.004.csv: 2e-11 <-- reference

In [26]:
# thesis folder
savefolder = '/Users/hkromer/polybox/Thesis/Chapters/DetailedNeutronGeneratorCharacterization/Figures/'
figname = 'COMSOL_mesh_refinement_temporal'
figname = f'{savefolder}/{figname}/{figname}.pdf'

In [27]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

In [28]:
# import data
path = '/Users/hkromer/02_PhD/02_Data/01_COMSOL/01_IonOptics/02.current_chamber/02.MR/temporal_refinement/data/temporal_resolution/time_outputs/'
d = {}
for ii in range(1,5):
    fname = f'{path}02.MR.102.temporal_refinement.save_1e8.00{ii}.csv'
    data = pd.read_csv(fname, skiprows=7)

    # remove the index
    data = data.iloc[:, 1:]

    # multicolumn indexing
    cols = data.columns.tolist()

    first_cols = [re.findall(r't=(.+)', c)[0] if len(re.findall(r't=(.+)', c)) > 0 else None for c in cols] # time
    second_cols = [re.findall(r'(.+) \(', c)[0] if len(re.findall(r't=(.+)', c)) > 0 else None for c in cols] # qx, qy, or qz

    tuples = [(first_index, second_index) for first_index, second_index in zip(first_cols, second_cols)]
    data.columns = pd.MultiIndex.from_tuples(tuples, names=['Time', 'Coordinate'])



    # for each time, compute the beam radius
    def compute_qr(df):
        """
        Compute beam radius for each particle trajectory
        """
        # select second level of columns, one as series
        y = df.xs('qy', axis=1, level=1).values
        z = df.xs('qz', axis=1, level=1).values

        r = np.sqrt(y**2+z**2)

#         y_0 = np.sum(y*r)/np.sum(r)
#         z_0 = np.sum(z*r)/np.sum(r)
        # y_0, z_0 = np.median(data_y), np.median(data_z)
        # create as a dataframe
        df_r = pd.DataFrame(np.sqrt((y)**2+(z)**2), columns=df.xs('qy', axis=1, level=1).columns)

        #create MultiIndex in columns 
        df_r.columns = pd.MultiIndex.from_product([df_r.columns, ['qr']])
        #join to original
        return pd.concat([df, df_r], axis=1).sort_index(axis=1)

    data = compute_qr(data)

    # select second level of columns, one as series
    data_qr = data.xs('qr', axis=1, level=1)

    data_x_ecdf = pd.DataFrame(np.sort(data_qr.T)).T
    data_x_ecdf.columns = data_qr.columns
    data_x_ecdf.columns = pd.MultiIndex.from_product([data_x_ecdf.columns, ['x_ecdf']])
    data = pd.concat([data, data_x_ecdf], axis=1).sort_index(axis=1)

    n = data_qr.shape[0]
    data_y_ecdf = pd.DataFrame([np.arange(1, n+1) / n for ii in range(data_qr.shape[1])], index=data_qr.columns).T
    data_y_ecdf.columns = pd.MultiIndex.from_product([data_y_ecdf.columns, ['y_ecdf']])
    data = pd.concat([data, data_y_ecdf], axis=1).sort_index(axis=1)

    # save data
    fname = f'{fname}cdf.csv'
    data.to_csv(fname)
    d[ii] = data
data

Time               0                                                 1.1E-7  \
Coordinate        qr   qx        qy        qz    x_ecdf    y_ecdf        qr   
0           0.552914 -1.4 -0.502726  0.230173  0.004584  0.000333  1.252589   
1           0.572977 -1.4 -0.516546  0.247958  0.012621  0.000667  1.292490   
2           0.566229 -1.4 -0.515293  0.234711  0.013451  0.001000  1.285187   
3           0.586433 -1.4 -0.529260  0.252561  0.020216  0.001333  1.323382   
4           0.593190 -1.4 -0.540721  0.243918  0.021413  0.001667  1.345961   
...              ...  ...       ...       ...       ...       ...       ...   
2995        0.287186 -1.4  0.257668 -0.126819  0.593244  0.998667  1.031079   
2996        0.305002 -1.4  0.279281 -0.122589  0.593247  0.999000  1.053767   
2997        0.291890 -1.4  0.266643 -0.118748  0.593258  0.999333  1.026423   
2998        0.283852 -1.4  0.262979 -0.106836  0.593370  0.999667  0.995249   
2999        0.289333 -1.4  0.271954 -0.098765  0.593411  1.000000  0.991778   

Time                                       ...      8E-8                      \
Coordinate         qx        qy        qz  ...        qy        qz    x_ecdf   
0           70.038821  1.245520 -0.132890  ...  1.245520 -0.132890  0.009557   
1           70.040949  1.279175 -0.185041  ...  1.279175 -0.185041  0.023038   
2           70.040804  1.276906 -0.145652  ...  1.276906 -0.145652  0.033336   
3           70.042848  1.308463 -0.198155  ...  1.308463 -0.198155  0.034817   
4           70.044590  1.334765 -0.173243  ...  1.334765 -0.173243  0.049516   
...               ...       ...       ...  ...       ...       ...       ...   
2995        70.008581 -0.585794  0.848510  ... -0.585794  0.848510  2.212661   
2996        70.010158 -0.637366  0.839159  ... -0.637366  0.839159  2.213892   
2997        70.009200 -0.606557  0.828030  ... -0.606557  0.828030  2.214343   
2998        70.008918 -0.597209  0.796154  ... -0.597209  0.796154  2.215993   
2999        70.009562 -0.618366  0.775401  ... -0.618366  0.775401  2.217020   

Time                      9E-8                                           \
Coordinate    y_ecdf        qr         qx        qy        qz    x_ecdf   
0           0.000333  1.252589  70.038821  1.245520 -0.132890  0.009557   
1           0.000667  1.292490  70.040949  1.279175 -0.185041  0.023038   
2           0.001000  1.285187  70.040804  1.276906 -0.145652  0.033336   
3           0.001333  1.323382  70.042848  1.308463 -0.198155  0.034817   
4           0.001667  1.345961  70.044590  1.334765 -0.173243  0.049516   
...              ...       ...        ...       ...       ...       ...   
2995        0.998667  1.031079  70.008581 -0.585794  0.848510  2.212661   
2996        0.999000  1.053767  70.010158 -0.637366  0.839159  2.213892   
2997        0.999333  1.026423  70.009200 -0.606557  0.828030  2.214343   
2998        0.999667  0.995249  70.008918 -0.597209  0.796154  2.215993   
2999        1.000000  0.991778  70.009562 -0.618366  0.775401  2.217020   

Time                  
Coordinate    y_ecdf  
0           0.000333  
1           0.000667  
2           0.001000  
3           0.001333  
4           0.001667  
...              ...  
2995        0.998667  
2996        0.999000  
2997        0.999333  
2998        0.999667  
2999        1.000000  

[3000 rows x 156 columns]

In [29]:
def compute_cdf(radii):
    """
    Takes an array of radii and computes the CDF from it. Returns the X and Y values of the cdf.
    """
    unique, radii_obs = np.unique(radii, return_counts=True)
    radii_obs = radii_obs / radii_obs.sum()  # Normalise

    cdf_radii = radii_obs.cumsum()
    
    x_cdf = unique
    y_cdf = cdf_radii
    
    return x_cdf, y_cdf

In [32]:
time = '1E-7'
radii = data[time, 'qr']
%matplotlib widget

x_cdf, y_cdf = compute_cdf(radii)
plt.plot(x_cdf, y_cdf, label="cheap cdf")
plt.plot(data[time, 'x_ecdf'].values, data[time, 'y_ecdf'].values, label="datacamp cdf")
plt.legend(loc=2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
# plot a random cdf
ii = np.random.randint(0,len(data_qr.columns))
time = data_qr.columns[ii]

print(time, data[time, 'x_ecdf'].name)
x = data[time, 'x_ecdf'].values
y = data[time, 'y_ecdf'].values

plt.plot(x, y)
plt.title(time)
plt.show()
t=data[time]
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    # Number of data points: n
    n = len(data)

    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y

r = t['qr'].values
x_ecdf, y_ecdf = ecdf(r)

plt.scatter(x_ecdf, y_ecdf)
plt.show()

0 ('0', 'x_ecdf')


# Compare the CDFs

- for each timestep, interpolate the CDFs of each of the 4 cases
- compare the CDFs based on Kolmogorov-Smirnov test, plot the overall largest deviation, or one example?

### Get a dictionary of the interpolations for each mesh

In [34]:
d_interp = {}
meshes = range(1,5)

def interpolate_CDF(col_x, df_y):
    """
    Takes a dataframe at one timestamp as input and returns an interpolation of this CDF.
    """
    time = col_x.name
    col_y = df_y.loc[:,time]

    interpolation = interpolate.interp1d(col_x.values, col_y.values, kind='slinear', fill_value="extrapolate")
    return interpolation

for ii in meshes:
    data = d[ii]

    # x and y values of the ecdf
    data_x_ecdf = data.xs('x_ecdf', axis=1, level=1)
    data_y_ecdf = data.xs('y_ecdf', axis=1, level=1)

    # interpolate 100000 datapoints for each timestamp
    # rows are the different times, cols are the 100000 x axis values (beam radii)
    data_interp_x = np.zeros([26,100000])
    data_interp_y = np.zeros([26,100000])

    # # same for the y values, which are the interpolated CDFs
    # data_interp_y = pd.DataFrame(0, index=data_y_ecdf.columns, columns=np.arange(0,100000,1))

    # minimal and maximal x value for the interpolation
    min_x = data_x_ecdf.min()
    max_x = data_x_ecdf.max()

    # get the query points for the interpolation
    data_interp_x = pd.DataFrame(np.linspace(min_x, max_x, 100000).T, index=data_x_ecdf.columns, columns=np.arange(0,100000,1))


    # interpolation for each time
    df_interp = data_x_ecdf.apply(lambda x: interpolate_CDF(x, data_y_ecdf))
    
    d_interp[ii] = df_interp



### Compute the absolute differences in the CDFs for each mesh and timestamp

In [35]:
# reference interpolated CDF
df_reference = d_interp[4]

# get also the query points from the reference CDF
data_ref = d[4]

# x and y values of the ecdf
data_x_ecdf = data_ref.xs('x_ecdf', axis=1, level=1)
data_y_ecdf = data_ref.xs('y_ecdf', axis=1, level=1)

# interpolate 100000 datapoints for each timestamp
# rows are the different times, cols are the 100000 x axis values (beam radii)
data_interp_x = np.zeros([26,100000])
data_interp_y = np.zeros([26,100000])

# # same for the y values, which are the interpolated CDFs
# data_interp_y = pd.DataFrame(0, index=data_y_ecdf.columns, columns=np.arange(0,100000,1))

# minimal and maximal x value for the interpolation
min_x = data_x_ecdf.min()
max_x = data_x_ecdf.max()

# get the query points for the interpolation
# rows are query points, columns are times
data_interp_x = pd.DataFrame(np.linspace(min_x, max_x, 100000).T, index=data_x_ecdf.columns, columns=np.arange(0,100000,1)).T

# dataframe to store the maximum differences
data_differences = pd.DataFrame(index=df_reference.index)

for ii in range(1,4):
    # get the interpolation
    data = d_interp[ii]
    data = data.reset_index()

    def compute_absolute_difference_in_CDF(interpolation, reference, querypoints):
        time = interpolation.loc['Time']
        interp = interpolation.loc[0]

        # reference interpolation
        interp_ref = reference.loc[time]

        # get the querypoints for this timestamp
        x = querypoints.loc[:,time].values

        # get the interpolated y values
        y_interpolated = interp(x)

        # get interpolated reference values
        y_interpolated_reference = interp_ref(x)

        # absolute difference between the CDFs
        diff_abs = np.abs(y_interpolated - y_interpolated_reference)
        s = pd.Series(diff_abs)

        s = s.append(pd.Series([time],index=['Time']))

        return s


    data_difference = data.apply(lambda x: compute_absolute_difference_in_CDF(x, df_reference, data_interp_x), axis=1).set_index('Time', drop=True)

    # maximum difference per timestep
    max_diffs = data_difference.max(axis=1)
    
    data_differences[ii] = max_diffs

In [36]:
data_differences

,1,2,3
Time,,,
0,0.026798,0.026798,0.026798
1.1E-7,0.006156,0.006977,0.007802
1.2E-7,0.006156,0.006977,0.007802
1.3E-7,0.006156,0.006977,0.007802
1.4E-7,0.006156,0.006977,0.007802
1.5E-7,0.006156,0.006977,0.007802
1.6E-7,0.006156,0.006977,0.007802
1.7E-7,0.006156,0.006977,0.007802
1.8E-7,0.006156,0.006977,0.007802


## Plot

In [37]:
# for run,c,m,linest in zip(runs, colors,markers,linestyles):
#     if run != '004':
#         data_plot = data_agg[data_agg['run'] == run]
# #         data_plot['time'] = data_plot['time'] * 1e6 # convert to mus
        
#         ts = data_plot['ts'].unique()[0]
#         Y = 100.0 * data_plot['rel_diff_qrmax'].values
#         ts = '-' + str(ts)
# #         ax1.plot(data_plot['x_avg'].values, Y, color=c, linestyle=linest, label=r'\textbf{{$2\times$10$^{{{}}}$}}'.format(ts), marker=m, markersize=markersize)
#         ax1.plot(data_plot['x_avg'].values, Y, color=c, linestyle=linest, label=r'\textbf{{$2\times$10$^{{{}}}$}}'.format(ts))

# ylims = ax1.get_ylim()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.ticker import AutoMinorLocator
import os

os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
matplotlib.rcParams['font.weight']= 'bold'
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Arial'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.tt'] = 'Arial'
matplotlib.rcParams['mathtext.cal'] = 'Arial'
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']
plt.rcParams["axes.labelweight"] = "heavy"


# thesis folder
savefolder = '/Users/hkromer/polybox/Thesis/Chapters/DetailedNeutronGeneratorCharacterization/Figures/'
figname = 'COMSOL_temporal_refinement'
figname = f'{savefolder}/{figname}/{figname}.pdf'


data_plot = data_differences.copy()

data_plot['timestamp'] = [float(t) for t in data_plot.index.values]
data_plot = data_plot.sort_values(by='timestamp')
data_plot = data_plot.set_index('timestamp', drop=True)

cols = data_plot.columns
colors = ['darkorange', 'darkblue', 'darkred']
linestyles = ['-', '--' , "dotted", (0, (3, 5, 1, 5))]
labels = [f"00{ii}" for ii in cols]

# mesh 1 2 3
# mesh_elements = [3800000, 830000, 670000]
mesh_elements = ['-10', '-9', '-8'] # time steps!!!
# mesh_elements = [1, 2, 3]

# plot for each time
fig, ax1 = plt.subplots(figsize=(8, 6))
fs = 20

for mesh, color, label, ls, ms in zip(cols, colors, labels, linestyles, mesh_elements):
    x = data_plot.index.values*1e6 # convert to mus
    y = data_plot.loc[:, mesh].values
    ax1.plot(x, y, color=color, linestyle=ls, linewidth=2.5, label=r'\textbf{{$2\times$10$^{{{}}}$}}'.format(ms))

ylims = ax1.get_ylim()


ax1.plot((0.0, 0.0), (-100, 100), 'k--', linewidth=2, color='black')  # Plasma meniscus
ax1.plot((0.08, 0.08), (-100, 100), 'k--', linewidth=2, color='black')  # Target surface
ax1.set_xlabel(r"\textbf{Simulation time [$\mu$s]}", fontsize=fs)
ax1.set_ylabel(r"\textbf{Maximum absolute difference in}" + '\n'+ r"\textbf{empirical CDF with respect to }" + r'\textbf{{$2\times$10$^{{{}}}$}}'.format('-11') + r'\textbf{s}', fontsize=fs)
ax1.tick_params('x', colors='black', labelsize=fs)
ax1.tick_params(axis='y', labelsize=fs-2)
ax1.set_ylim(ylims[0],ylims[1])
ax1.text(-0.0035, 0.065, s=r'\textbf{Plasma meniscus}', color='black',  fontsize=fs-4, rotation=90) # PLASMA
ax1.text(0.076, 0.065, s=r'\textbf{Target surface}', color='black',  fontsize=fs-4, rotation=90) # target


# ax1.set_yticks(np.arange(2.0,3.1,0.2))
# minor ticks x
minor_locator = AutoMinorLocator(2)
ax1.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax1.yaxis.set_minor_locator(minor_locator)

hs, ls = ax1.get_legend_handles_labels()

leg1 = plt.legend([hs[2],hs[1],hs[0]],[ls[2],ls[1],ls[0]], loc=(0.53,0.73), fontsize=fs-6)
# leg1 = plt.legend(loc=(0.69,0.74), fontsize=fs-6)
# leg1 = plt.legend(loc='best', fontsize=fs-6)
leg1.set_title(r"\textbf{Time stepping}", prop = {'size': fs-4})
ax1.grid(b=True, which='major', linestyle='-')
ax1.grid(b=True, which='minor', linestyle='--')

plt.ylim(0.0, 0.1)
plt.xlim(-0.005, 0.1)

plt.tight_layout()
# plt.savefig(figname)
plt.show()


## Check and plot the interpolation

In [ ]:
# pick one time at random
data_qr = d[1].xs('qr', axis=1, level=1)
ii = np.random.randint(0,len(data_qr.columns))
time = data_qr.columns[ii]

data = d[1][time]
interp = interpolate.interp1d(data['x_ecdf'].values, data['y_ecdf'].values, kind='slinear')

x = np.linspace(data['x_ecdf'].min(), data['x_ecdf'].max(), 100000)

fig, ax = plt.subplots(figsize=(10,5))

plt.title(time)
plt.plot(x, interp(x), color='red')
plt.plot(data['x_ecdf'].values, data['y_ecdf'].values)
plt.show()

fig, ax = plt.subplots(figsize=(10,5))

plt.title("interpolated df")
plt.plot(x, df_interp.loc[time](x), color='red')

plt.show()

# fig, ax = plt.subplots(figsize=(20,10))

# plt.title(time)
# plt.plot(x, interp(x), '-o', color='red')
# plt.plot(data['x_ecdf'].values, data['y_ecdf'].values)

# plt.xlim(1,1.025)
# plt.ylim(0.42,0.445)
# plt.show()


# Plot two ECDFs to show the KS Test

In [ ]:
import numpy as np
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx], idx

find_nearest(x_ecdf_ref, 0.12)
find_nearest(x_ecdf_ref, 0.12), x_ecdf_ref[2534],y_ecdf_ref[2534]

In [ ]:
# thesis folder
savefolder = '/Users/hkromer/polybox/Thesis/Chapters/DetailedNeutronGeneratorCharacterization/Figures/'
figname = 'COMSOL_mesh_refinement_two_ECDFs'
figname = f'{savefolder}/{figname}/{figname}.pdf'

# select time
time = '3E-8'

x_ecdf3 = d[3].xs('x_ecdf', axis=1, level=1).loc[:, time].values
x_ecdf_ref = d[4].xs('x_ecdf', axis=1, level=1).loc[:, time].values

y_ecdf3 = d[3].xs('y_ecdf', axis=1, level=1).loc[:, time].values
y_ecdf_ref = d[4].xs('y_ecdf', axis=1, level=1).loc[:, time].values



# plot 
fig, ax1 = plt.subplots(figsize=(8, 6))
fs = 20

plt.plot(x_ecdf3, y_ecdf3, color='darkblue', linewidth=2.5, label='A')
plt.plot(x_ecdf_ref, y_ecdf_ref, '--', color='darkorange', linewidth=2.5, label='D')

ylims = ax1.get_ylim()


# ax1.plot((0.0, 0.0), (-100, 100), 'k--', linewidth=2, color='black')  # Plasma meniscus
# ax1.plot((0.08, 0.08), (-100, 100), 'k--', linewidth=2, color='black')  # Target surface
ax1.set_xlabel(r"$\sqrt{\textbf{y}^{\textrm{\textbf{2}}}-\textbf{z}^{\textrm{\textbf{2}}}}$"+ r" \textbf{[mm]}", fontsize=fs)
ax1.set_ylabel(r"\textbf{Empirical CDF [-]}", fontsize=fs)
ax1.tick_params('x', colors='black', labelsize=fs)
ax1.tick_params(axis='y', labelsize=fs-2)
ax1.set_ylim(ylims[0],ylims[1])
# ax1.text(-0.0035, 0.065, s=r'\textbf{Plasma meniscus}', color='black',  fontsize=fs-4, rotation=90) # PLASMA
# ax1.text(0.076, 0.065, s=r'\textbf{Target surface}', color='black',  fontsize=fs-4, rotation=90) # target


# ax1.set_yticks(np.arange(2.0,3.1,0.2))
# minor ticks x
minor_locator = AutoMinorLocator(2)
ax1.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax1.yaxis.set_minor_locator(minor_locator)

hs, ls = ax1.get_legend_handles_labels()

# leg1 = plt.legend([hs[2],hs[1],hs[0]],[ls[2],ls[1],ls[0]], loc=(0.53,0.73), fontsize=fs-6)
# leg1 = plt.legend(loc=(0.69,0.74), fontsize=fs-6)
leg1 = plt.legend(loc='best', fontsize=fs-6)
leg1.set_title(r"\textbf{Mesh}", prop = {'size': fs-4})
ax1.grid(b=True, which='major', linestyle='-')
ax1.grid(b=True, which='minor', linestyle='--')
ax1.plot([0.12, 0.12], [0.845, 0.9316666666666666], color='darkred', linewidth=3.5)
offset = 0.00125
ax1.plot([0.12-offset, 0.12+offset], [0.845, 0.845], color='darkred', linewidth=3.5)
ax1.plot([0.12-offset, 0.12+offset], [0.9316666666666666, 0.9316666666666666], color='darkred', linewidth=3.5)
ax1.text(0.074, 0.865, r'\textbf{Maximum difference}', color='darkred', fontsize=16)
plt.ylim(0.0, 1.0)
plt.xlim(0, 0.14)

plt.tight_layout()
plt.savefig(figname)

plt.show()

# Plot 2D some particles

In [ ]:
time = '2.5E-7'
# take 100 random samples 
number_of_particles = 300 
rand_numbers = np.random.randint(0, high=d[4].shape[0], size=number_of_particles)

In [ ]:
# compute weight centre of the beam
data_y = d[4].xs('qy', axis=1, level=1).loc[:, time].values.astype(np.float64)
data_z = d[4].xs('qz', axis=1, level=1).loc[:, time].values.astype(np.float64)

r = np.sqrt(data_y**2+data_z**2)
y_0 = np.sum(data_y*r)/np.sum(r)
z_0 = np.sum(data_z*r)/np.sum(r)
# y_0, z_0 = np.median(data_y), np.median(data_z)

In [ ]:
savefolder = '/Users/hkromer/polybox/Thesis/Chapters/DetailedNeutronGeneratorCharacterization/Figures/'
figname = 'COMSOL_mesh_refinement_beam_radius'
figname = f'{savefolder}/{figname}/{figname}.pdf'

data_y = d[4].xs('qy', axis=1, level=1).loc[rand_numbers, time].values.astype(np.float64)
data_z = d[4].xs('qz', axis=1, level=1).loc[rand_numbers, time].values.astype(np.float64)

fig, ax = plt.subplots(figsize=(10,10))
s0 = 280
ax.scatter(data_y, data_z, marker='x', color='red', s=s0)
ax.scatter(0, 0, marker='o', color='darkgreen', s=s0+3*s0)
# ax.patch.set_facecolor('blue')
ax.set_xticks([ ])
ax.set_yticks([ ])
ax.set_facecolor('#FFFFEB')
plt.axis('equal')
plt.tight_layout()
plt.savefig(figname)
plt.show()

In [ ]:
import numpy as np

def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    # Number of data points: n
    n = len(data)

    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y
ecdf([3, 3, 1, 4])

In [ ]:
ecdf([3, 55, 0.5, 1.5])